# Handoff Pattern in Agent Orchestration

This notebook demonstrates the handoff pattern for agent orchestration using Semantic Kernel and Azure OpenAI. The handoff pattern allows tasks to be transferred between agents based on specific triggers or conditions, ensuring efficient task completion.

In [3]:
# Install Semantic Kernel and other required libraries
%pip install semantic-kernel

# Import necessary libraries
from semantic_kernel.agents import ChatCompletionAgent, OrchestrationHandoffs, HandoffOrchestration
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.functions import kernel_function

Note: you may need to restart the kernel to use updated packages.


**Expected Output**: Package installation messages will appear, followed by successful import confirmation. No output will be shown if all imports are successful.

In [ ]:
# Define plugins for specific tasks

class OrderStatusPlugin:
    @kernel_function
    def check_order_status(self, order_id: str) -> str:
        """Check the status of an order."""
        return f"Order {order_id} is shipped and will arrive in 2-3 days."

class OrderRefundPlugin:
    @kernel_function
    def process_refund(self, order_id: str, reason: str) -> str:
        """Process a refund for an order."""
        print(f"Processing refund for order {order_id} due to: {reason}")
        return f"Refund for order {order_id} has been processed successfully."

class OrderReturnPlugin:
    @kernel_function
    def process_return(self, order_id: str, reason: str) -> str:
        """Process a return for an order."""
        print(f"Processing return for order {order_id} due to: {reason}")
        return f"Return for order {order_id} has been processed successfully."

# Configure Azure OpenAI service
api_key = "**"
endpoint = "***"
deployment_name = "****"

openai_service = AzureChatCompletion(
    api_key="YOUR_AZURE_OPENAI_API_KEY",
    endpoint="YOUR_AZURE_OPENAI_ENDPOINT",
    deployment_name="YOUR_DEPLOYMENT_NAME"
)

# Define specialized agents
support_agent = ChatCompletionAgent(
    name="TriageAgent",
    description="A customer support agent that triages issues.",
    instructions="Handle customer requests and route them to appropriate specialists.",
    service=openai_service,
)

refund_agent = ChatCompletionAgent(
    name="RefundAgent",
    description="A customer support agent that handles refunds.",
    instructions="Handle refund requests and process refunds.",
    service=openai_service,
    plugins=[OrderRefundPlugin()],
)

order_status_agent = ChatCompletionAgent(
    name="OrderStatusAgent",
    description="A customer support agent that checks order status.",
    instructions="Handle order status requests and provide tracking information.",
    service=openai_service,
    plugins=[OrderStatusPlugin()],
)

order_return_agent = ChatCompletionAgent(
    name="OrderReturnAgent",
    description="A customer support agent that handles order returns.",
    instructions="Handle order return requests and process returns.",
    service=openai_service,
    plugins=[OrderReturnPlugin()],
)

**Expected Output**: No output will be displayed as this cell only defines the plugin classes and functions. These will be used by agents later in the handoff orchestration.

In [5]:
# Define handoff relationships between agents

handoffs = (
    OrchestrationHandoffs()
    .add_many(  # Add multiple handoffs from the triage agent
        source_agent=support_agent.name,
        target_agents={
            refund_agent.name: "Transfer to this agent if the issue is refund related",
            order_status_agent.name: "Transfer to this agent if the issue is order status related",
            order_return_agent.name: "Transfer to this agent if the issue is order return related",
        },
    )
    .add(  # Add individual handoffs back to triage agent
        source_agent=refund_agent.name,
        target_agent=support_agent.name,
        description="Transfer to this agent if the issue is not refund related",
    )
    .add(
        source_agent=order_status_agent.name,
        target_agent=support_agent.name,
        description="Transfer to this agent if the issue is not order status related",
    )
    .add(
        source_agent=order_return_agent.name,
        target_agent=support_agent.name,
        description="Transfer to this agent if the issue is not order return related",
    )
)

# Define callback function to observe agent responses
def agent_response_callback(message: ChatMessageContent) -> None:
    print(f"{message.name}: {message.content}")

# Define human response function for interactive input
def human_response_function() -> ChatMessageContent:
    user_input = input("User: ")
    return ChatMessageContent(role=AuthorRole.USER, content=user_input)

**Expected Output**: No output will be displayed as this cell creates the agent instances. The agents are now ready to be used in the handoff orchestration system.

In [6]:
# Set up the handoff orchestration

handoff_orchestration = HandoffOrchestration(
    members=[
        support_agent,
        refund_agent,
        order_status_agent,
        order_return_agent,
    ],
    handoffs=handoffs,
    agent_response_callback=agent_response_callback,
    human_response_function=human_response_function,
)

# Start the runtime
runtime = InProcessRuntime()
runtime.start()

print("Handoff orchestration setup complete!")

Handoff orchestration setup complete!


TriageAgent: Hello! You've reached the Triage Agent. How can I assist you today?
OrderStatusAgent: Hello! You've reached the Order Status Agent. Could you please provide me with the order ID so I can check the status for you?
OrderStatusAgent: Your order with ID 3456789 has been shipped and is expected to arrive in 2–3 days. Is there anything else I can assist you with?


**Expected Output**: Confirmation messages showing that the handoff orchestration is configured and the runtime is started. You'll see status messages about the setup being complete.

In [ ]:
# Execute the handoff orchestration

print("Starting handoff orchestration...")
print("You can interact with the agents by typing your requests.")
print("Try asking about order status, refunds, or returns!")

# Invoke the orchestration with initial task
orchestration_result = await handoff_orchestration.invoke(
    task="A customer is on the line.",
    runtime=runtime,
)

# Get the final result
value = await orchestration_result.get()
print(f"\nFinal Result: {value}")

# Stop the runtime when done
await runtime.stop_when_idle()
print("Orchestration completed!")

Starting handoff orchestration...
You can interact with the agents by typing your requests.
Try asking about order status, refunds, or returns!


**Expected Output**: The handoff orchestration will execute, showing the task being processed and how control is transferred between different agents based on the task requirements. You'll see the conversation flow and agent responses.

## Simulated Conversation Example

For demonstration purposes, here's how you can simulate a conversation with predefined responses instead of interactive input:

In [ ]:
# Alternative: Simulated conversation with predefined responses
from collections import deque

# Simulate user input with a queue of responses
responses = deque([
    "I'd like to track the status of my order",
    "My order ID is 123", 
    "I want to return another order of mine",
    "Order ID 321",
    "Broken item",
    "No, bye"
])

def simulated_human_response_function() -> ChatMessageContent:
    if responses:
        user_input = responses.popleft()
        print(f"User: {user_input}")
        return ChatMessageContent(role=AuthorRole.USER, content=user_input)
    else:
        return ChatMessageContent(role=AuthorRole.USER, content="No, bye")

# Create orchestration with simulated responses
simulated_orchestration = HandoffOrchestration(
    members=[
        support_agent,
        refund_agent, 
        order_status_agent,
        order_return_agent,
    ],
    handoffs=handoffs,
    agent_response_callback=agent_response_callback,
    human_response_function=simulated_human_response_function,
)

# Run the simulated conversation
print("\n=== Simulated Handoff Conversation ===")
orchestration_result = await simulated_orchestration.invoke(
    task="A customer is on the line.",
    runtime=runtime,
)

# Get the final result
value = await orchestration_result.get()
print(f"\nTask completed with summary: {value}")

**Expected Output**: Results from the alternative conversation simulation, showing predefined responses demonstrating how the handoff pattern works with different types of queries and agent transitions.